#  Exercise-correlated Knowledge Proficiency Tracing model (EKPT)

This notebook will show you how to train and use the EKPT.
First, we will show how to get the data (here we use assistment-2009-2010-skill as the dataset).
Then we will show how to train an EKPT, and perform the parameters persistence.
At last, we will show how to load the parameters from the file and evaluate on the test dataset.

The script version could be found in [EKPT.py](EKPT.ipynb)

## Data Preparation

Before we process the data, we need to first acquire the dataset which is shown in [prepare_dataset.ipynb](prepare_dataset.ipynb)

In [1]:
# Load the data from files
import numpy as np
import json

# Q matrix
q_m = np.loadtxt("../../data/2009_skill_builder_data_corrected/q_m.csv", dtype=int, delimiter=",")
prob_num, know_num = q_m.shape[0], q_m.shape[1]

# training data
with open("../../data/2009_skill_builder_data_corrected/train_data.json", encoding='utf-8') as file:
    train_set = json.load(file)
stu_num = max([x['user_id'] for x in train_set[0]]) + 1
time_window_num = len(train_set)
                    
# testing data
with open("../../data/2009_skill_builder_data_corrected/test_data.json", encoding='utf-8') as file:
    test_set = json.load(file)

In [2]:
# train_set[0][0] is the first train log in the first Time Window
print(train_set[0][0], test_set[0])

{'user_id': 152, 'item_id': 2515, 'score': 1} {'user_id': 21, 'item_id': 5152, 'score': 1}


In [3]:
len(train_set), len(test_set)

(7, 16266)

## Training and Persistence

In [4]:
import logging
logging.getLogger().setLevel(logging.INFO)

In [5]:
from EduKTM import EKPT

cdm = EKPT(q_m, stu_num, prob_num, know_num, time_window_num=time_window_num)

cdm.train(train_set, epoch=2, lr=0.001, lr_b=0.0001, epsilon=1e-3, init_method='mean')
cdm.save("ekpt.params")

INFO:root:save parameters to ekpt.params


## Loading and Testing

In [6]:
cdm.load("ekpt.params")
rmse, mae = cdm.eval(test_set)
print("For EKPT, RMSE: %.6f, MAE: %.6f" % (rmse, mae))

INFO:root:load parameters from ekpt.params
evaluating: 100%|█████████████████████████████████████████████████████████████| 16266/16266 [00:00<00:00, 29167.17it/s]

For EKPT, RMSE: 0.448456, MAE: 0.380642
